<a href="https://colab.research.google.com/github/samitha278/miniVGG/blob/main/build_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

## Simple Convolution function

In [33]:
def Convolution(image,kernel):

  x,y = image.shape

  a,b = kernel.shape

  feature_map = []

  for i in range(y-b):

    row = []

    for j in range(x-a):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [29]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel)


In [32]:
feature_map.shape

torch.Size([29, 29])

## Convolution with stride

In [36]:
def Convolution(image,kernel,stride=(1,1)):

  x,y = image.shape
  a,b = kernel.shape

  r,c = stride

  feature_map = []

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [37]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,1))      # 2: aloge side rows , 1: along side columns


In [39]:
feature_map.shape

torch.Size([29, 15])